# Régression probabiliste

Ce notebook est inspiré par les notes de cours de Fabrice Gamboa et des planches de Carl Edward Rasmussen et Zoubin Ghahramani : 
* https://mlg.eng.cam.ac.uk/teaching/4f13/1314/lect0102.pdf
* https://mlg.eng.cam.ac.uk/teaching/4f13/1314/lect0304.pdf

### 1. Rappel de la régression linéaire

On suppose la structure de modèle $\mathcal{M}$ suivante : 
\begin{align*}
    y_i & = \beta_0 + \beta_1 x_i^1 + \beta_2 x_i^2 + \ldots + \beta_d x_i^d + \epsilon_i; & \epsilon_i \sim \mathcal{N}(0, \sigma^2)\\
                          & = \boldsymbol{\phi}(x_i)^T \boldsymbol{\beta} + \epsilon_i & 
\end{align*}
où 
$$\boldsymbol{\phi}(x_i) = \begin{bmatrix} 1 \\ x_i \\ x_i^2 \\ \vdots \\ x_i^d \end{bmatrix} \:\:\:\:\:\:\:\: et \:\:\:\:\:\:\:\: \boldsymbol{\beta} = \begin{bmatrix} \beta_0 \\ \beta_1 \\ \beta_2 \\ \vdots \\ \beta_d \end{bmatrix}$$

Notons $\boldsymbol{x} = [x_1 \ldots x_N]^T$ les variables explicatives et $\boldsymbol{y} = [y_1 \ldots y_N]^T$ les variables à prédire de la base d'apprentissage qui contient $N$ échantillons. 

On définit la matrice $\boldsymbol{\phi}$ comme suit :  
$$\boldsymbol{\phi} = \begin{bmatrix} \boldsymbol{\phi}(x_1)^T \\ \vdots \\ \boldsymbol{\phi}(x_N)^T\end{bmatrix}$$

Par l'hypothèse de normalité, on rappelle que la vraisemblance $p(\boldsymbol{y} \vert \boldsymbol{x}, \boldsymbol{\beta}, \mathcal{M})$ est définie comme suit :
\begin{align*}
    p(\boldsymbol{y} \vert \boldsymbol{x}, \boldsymbol{\beta}, \mathcal{M}) = \mathcal{N}(\boldsymbol{y}; \boldsymbol{\phi}\boldsymbol{\beta}, \sigma^2\boldsymbol{I})
\end{align*}

On obtient l'estimation des paramètres du modèle par la méthode du maximum vraisemblance :
$$ \hat{\boldsymbol{\beta}} = \underset{\boldsymbol{\beta}}{\operatorname{argmax}} p(\boldsymbol{y} \vert \boldsymbol{x}, \boldsymbol{\beta}, \mathcal{M})$$

In [ ]:
import numpy as np; np.random.seed(3)
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

In [ ]:
n = 10
x_train = 3 * np.random.rand(n) - 1.5
y_train = 0.5 * x_train - 0.08 * x_train**2 - 0.05 * x_train**3 + 0.5 * np.random.rand(n)

In [ ]:
class LinearModel:
    """
    A Python class that defines a linear model for regression.
    """
    def __init__(self, degree):
        self.degree = degree

    @staticmethod
    def polynomial_features(X, degree):
        features = [X**k for k in range(degree+1)]
        features = np.stack(features, axis=1)
        return features

    def fit(self, X, y):
        X = self.polynomial_features(X, self.degree)
        XTX = np.matmul(X.T, X)
        assert np.linalg.det(XTX) != 0
        self.beta_hat = np.linalg.inv(XTX).dot(X.T).dot(y)

    def pred(self, X):
        X = self.polynomial_features(X, self.degree)
        y_hat = X.dot(self.beta_hat)
        return y_hat

In [ ]:
# Optimize linear models w/ various polynomial degrees
models = []
degrees = [0, 1, 2, 6]

for degree in degrees:
    model = LinearModel(degree=degree)
    model.fit(x_train, y_train)
    models.append(model)

# Make predictions over the training range
x_test = np.linspace(x_train.min() - 0.1, x_train.max() + 0.1, 100)
y_preds = []
for model in models:
    y_preds.append(model.pred(x_test))

# Plot the predictions
colors = sns.color_palette("husl", len(degrees))
fig, ax = plt.subplots(1, 4, figsize=(12, 2))

for i, degree in enumerate(degrees):
    ax[i].scatter(x_train, y_train)
    ax[i].plot(x_test, y_preds[i], color=colors[i])
    ax[i].set_title('Degree: {}'.format(degree))
    ax[i].set_xlabel('x')
    if i == 0:
        ax[i].set_ylabel(r'$\hat{y}$')
plt.show()

### 2. Régression probabiliste : une approche Bayésienne

On change de paradigme : on suppose qu'il n'existe plus un seul jeu de paramètres à estimer, mais que plusieurs paramètres sont vraisemblables.

Supposons la structure de modèle $\mathcal{M}$ suivante : 
\begin{align*}
    y_i & = \beta_0 + \beta_1 x_i^1 + \beta_2 x_i^2 + \ldots + \beta_6 x_i^6 + \epsilon_i; & \epsilon_i \sim \mathcal{N}(0, \sigma^2)\\
                          & = \boldsymbol{\phi}(x_i)^T \boldsymbol{\beta} + \epsilon_i & 
\end{align*}
Formulons un *a priori* sur la distribution des paramètres : 
$$p(\boldsymbol{\beta} \vert \mathcal{M}) = \mathcal{N}(\boldsymbol{0}, \sigma_\beta^2 \boldsymbol{I})$$
Sans prendre en compte les observations, on peut générer des prédictions à partir des paramètres du modèle qui sont *a priori* vraisemblables.

In [ ]:
class ProbabilisticLinearModel:
    """
    A Python class that defines a linear model for regression.
    """
    def __init__(self, degree, prior_mean=0, prior_std=1):
        self.degree = degree
        self.prior_mean = prior_mean
        self.prior_std = prior_std

    @staticmethod
    def polynomial_features(X, degree):
        features = [X**k for k in range(degree+1)]
        features = np.stack(features, axis=1)
        return features

    def sample_prior(self):
        beta_prior = np.random.normal(
            self.prior_mean * np.ones(self.degree + 1), 
            self.prior_std**2 * np.ones(self.degree + 1)
        )
        return beta_prior

    def pred(self, X, fitted=True):
        X = self.polynomial_features(X, self.degree)
        if fitted:
            beta_hat = self.sample_posterior()
        else:
            beta_hat = self.sample_prior()
        y_hat = X.dot(beta_hat)
        return y_hat

In [ ]:
prob_model = ProbabilisticLinearModel(degree=8, prior_mean=0, prior_std=1)
n_samples = 4
# Make predictions over the training range
x_test = np.linspace(x_train.min() - 0.1, x_train.max() + 0.1, 100)
y_preds = []

for i in range(n_samples):
    y_preds.append(prob_model.pred(x_test, fitted=False))

# Plot the predictions
colors = sns.color_palette("husl", n_samples)
fig, ax = plt.subplots(1, 4, figsize=(12, 2))

for i in range(n_samples):
    ax[i].plot(x_test, y_preds[i], color=colors[i])
    ax[i].set_title('Degree: {}'.format(prob_model.degree))
    ax[i].set_xlim(x_test[0], x_test[-1])
    ax[i].set_ylim(-4, 4)
    ax[i].set_xlabel('x')
    if i == 0:
        ax[i].set_ylabel(r'$\hat{y}$')
plt.show()

Etant donné les observations, on va maintenant "ajuster" les paramètres du modèle.

D'après la loi de Bayes, on a :
\begin{align*}
    p( \boldsymbol{\beta} \vert  \boldsymbol{y}, \boldsymbol{x},\mathcal{M}) = \frac{p(\boldsymbol{y} \vert \boldsymbol{x}, \boldsymbol{\beta}, \mathcal{M})p(\boldsymbol{\beta}, \mathcal{M})}{p(\boldsymbol{y} \vert \boldsymbol{x}, \mathcal{M})}
\end{align*}
où $p(\boldsymbol{y} \vert \boldsymbol{x}, \mathcal{M}) = \int p(\boldsymbol{y}, \boldsymbol{\beta} \vert \boldsymbol{x}, \mathcal{M})d\boldsymbol{\beta} = \int  p(\boldsymbol{y} \vert \boldsymbol{x}, \boldsymbol{\beta}, \mathcal{M})p(\boldsymbol{\beta}, \mathcal{M}) d\boldsymbol{\beta}$

De plus, comme la distribution de vraisemblance et la distribution *a priori* sont gaussiennes, la distribution *a posteriori* est également gaussienne : 
\begin{align*}
    p( \boldsymbol{\beta} \vert  \boldsymbol{y}, \boldsymbol{x}, \mathcal{M}) & = \mathcal{N}(\boldsymbol{\beta} ; \boldsymbol{\mu}, \boldsymbol{\Sigma)} \\
    \mbox{avec } \boldsymbol{\mu} = (\boldsymbol{\phi}^T\boldsymbol{\phi} + \frac{\sigma^2}{\sigma_{\beta}^2} \boldsymbol{I})^{-1}\boldsymbol{\phi}^T\boldsymbol{y} \:\:\:\:\: & \mbox{ et }  \:\:\:\:\: \boldsymbol{\Sigma} = (\sigma^{-2}\boldsymbol{\phi}^T\boldsymbol{\phi} + \sigma_\beta^{-2}\boldsymbol{I})^{-1}
\end{align*}

Ici, on n'obtient pas d'estimation ponctuelle, *e.g.* du maximum de vraisemblance. A la place, le loi de Bayes nous donne une distribution *a posteriori* sur les paramètres du modèle. Elle définit les paramètres qui sont vraisemblables compte tenu des données. Ci-dessous, on échantillonne plusieurs paramètres $\boldsymbol{\beta}$ selon la distribution *a posteriori*.

In [ ]:
class ProbabilisticLinearModel:
    """
    A Python class that defines a probabilistic linear model for regression.
    """
    def __init__(self, degree, prior_mean=0, prior_std=1):
        self.degree = degree
        self.prior_mean = prior_mean
        self.prior_std = prior_std

    @staticmethod
    def polynomial_features(X, degree):
        features = [X**k for k in range(degree+1)]
        features = np.stack(features, axis=1)
        return features

    def sample_prior(self):
        beta_prior = np.random.normal(
            self.prior_mean * np.ones(self.degree + 1), 
            self.prior_std**2 * np.ones(self.degree + 1)
        )
        return beta_prior

    def pred(self, X, fitted=True):
        X = self.polynomial_features(X, self.degree)
        if fitted:
            beta_hat = self.sample_posterior()
        else:
            beta_hat = self.sample_prior()
        y_hat = X.dot(beta_hat)
        return y_hat

    def estimate_noise(self, X, y):
        X = self.polynomial_features(X, self.degree)
        XTX = np.matmul(X.T, X)
        assert np.linalg.det(XTX) != 0
        beta_hat = np.linalg.inv(XTX).dot(X.T).dot(y)
        y_hat = X.dot(beta_hat)
        noise = np.sum((y - y_hat)**2) / (X.shape[0] - X.shape[1])
        return noise
        

    def fit(self, X, y):
        self.noise = self.estimate_noise(X, y)
        X = self.polynomial_features(X, self.degree)
        XTX = np.matmul(X.T, X)
        assert np.linalg.det(XTX) != 0
        self.mean_posterior = np.linalg.inv(
            XTX + self.noise / (self.prior_std**2) * np.eye(X.shape[1])
        ).dot(X.T).dot(y)
        self.cov_posterior = np.linalg.inv(
            XTX / self.noise + (1 / (self.prior_std**2)) * np.eye(X.shape[1])
        )

    def sample_posterior(self):
        return np.random.multivariate_normal(self.mean_posterior, self.cov_posterior)

In [ ]:
prob_model = ProbabilisticLinearModel(degree=5, prior_mean=0, prior_std=1)
prob_model.fit(x_train, y_train)

In [ ]:
# Make predictions over the training range
y_preds = []
n_samples = 4
for i in range(n_samples):
    y_preds.append(prob_model.pred(x_test, fitted=True))

# Plot the predictions
colors = sns.color_palette("husl", n_samples)
fig, ax = plt.subplots(1, 4, figsize=(12, 2))

for i in range(n_samples):
    ax[i].scatter(x_train, y_train)
    ax[i].plot(x_test, y_preds[i], color=colors[i])
    ax[i].set_title('Degree: {}'.format(prob_model.degree))
    ax[i].set_xlabel('x')
    if i == 0:
        ax[i].set_ylabel(r'$\hat{y}$')
plt.show()

On peut calculer la moyenne et l'écart-type empirique de plusieurs prédictions réalisées à partir de différents paramètres.

In [ ]:
# Make predictions over the training range
x_test = np.linspace(x_train.min() - 0.3, x_train.max() + 0.3, 100)
y_preds = []
n_samples = 100
for i in range(n_samples):
    y_preds.append(prob_model.pred(x_test, fitted=True))
y_preds = np.stack(y_preds)
mean_pred = np.mean(y_preds, axis=0)
std_pred = np.std(y_preds, axis=0)

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x_train, y_train)
mean = ax.plot(x_test, mean_pred, color='black', label='Moyenne')
ax.fill_between(x_test, mean_pred + std_pred, mean_pred - std_pred, alpha=0.2, color='orange')
for i in range(n_samples):
    ax.plot(x_test, y_preds[i], alpha=0.04, color='black')
patch = mpatches.Patch(color='orange', alpha=0.2, label=r'Moyenne $\pm$ écart-type')
plt.legend(handles=[mean[0], patch])
ax.set_xlabel('x')
ax.set_ylabel(r'$\hat{y}$')
plt.show()

Mais dans le cas où l'on modélise la distribution *a priori* et la vraisemblance par des Gaussiennes, on peut calculer analytiquement la distribution conditionnelle de la variable à prédire sachant la variable explicative, les données d'apprentissage, et la structure du modèle :
$$p(y^\ast \vert x^\ast, \boldsymbol{x}, \boldsymbol{y}, \mathcal{M}) = \mathcal{N}\big(y^\ast; \boldsymbol{\phi}(x^\ast)^T\boldsymbol{\mu}, \boldsymbol{\phi}(x^\ast)^T \boldsymbol{\Sigma} \boldsymbol{\phi}(x^\ast) + \sigma^2\big) $$

In [ ]:
class ProbabilisticLinearModel:
    """
    A Python class that defines a probabilistic linear model for regression.
    """
    def __init__(self, degree, prior_mean=0, prior_std=1):
        self.degree = degree
        self.prior_mean = prior_mean
        self.prior_std = prior_std

    @staticmethod
    def polynomial_features(X, degree):
        features = [X**k for k in range(degree+1)]
        features = np.stack(features, axis=1)
        return features

    def sample_prior(self):
        beta_prior = np.random.normal(
            self.prior_mean * np.ones(self.degree + 1), 
            self.prior_std**2 * np.ones(self.degree + 1)
        )
        return beta_prior

    def pred(self, X, fitted=True):
        X = self.polynomial_features(X, self.degree)
        if fitted:
            beta_hat = self.sample_posterior()
        else:
            beta_hat = self.sample_prior()
        y_hat = X.dot(beta_hat)
        return y_hat

    def estimate_noise(self, X, y):
        X = self.polynomial_features(X, self.degree)
        XTX = np.matmul(X.T, X)
        assert np.linalg.det(XTX) != 0
        beta_hat = np.linalg.inv(XTX).dot(X.T).dot(y)
        y_hat = X.dot(beta_hat)
        noise = np.sum((y - y_hat)**2) / (X.shape[0] - X.shape[1])
        return noise
        

    def fit(self, X, y):
        self.noise = self.estimate_noise(X, y)
        X = self.polynomial_features(X, self.degree)
        XTX = np.matmul(X.T, X)
        assert np.linalg.det(XTX) != 0
        self.mean_posterior = np.linalg.inv(
            XTX + self.noise / (self.prior_std**2) * np.eye(X.shape[1])
        ).dot(X.T).dot(y)
        self.cov_posterior = np.linalg.inv(
            XTX / self.noise + (1 / (self.prior_std**2)) * np.eye(X.shape[1])
        )

    def sample_posterior(self):
        return np.random.multivariate_normal(self.mean_posterior, self.cov_posterior)

    def pred_mean(self, x_star):
        x = self.polynomial_features(x_star, self.degree)
        return x.dot(self.mean_posterior)
        
    def pred_interval(self, x_star, p_int=0.95):
        x = self.polynomial_features(x_star, self.degree)
        mean = x.dot(self.mean_posterior)
        cov = np.matmul(x, self.cov_posterior)[:, np.newaxis, :]
        cov = np.matmul(cov, x[:, :, np.newaxis]).reshape(-1)
        Q = norm.isf((1-p_int) / 2)
        l = Q * np.sqrt(cov + self.noise)
        low = mean - l
        high = mean + l
        return (low, high)

In [ ]:
prob_model = ProbabilisticLinearModel(degree=6, prior_mean=0, prior_std=1)
prob_model.fit(x_train, y_train)

# Make predictions over the training range
x_test = np.linspace(x_train.min() - 0.3, x_train.max() + 0.3, 100)
pred_mean = prob_model.pred_mean(x_test)
p_int = 0.95
low, high = prob_model.pred_interval(x_test, p_int=p_int)

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x_train, y_train)
mean = ax.plot(x_test, pred_mean, color='black', label='Espérance')
ax.fill_between(x_test, low, high, alpha=0.2, color='orange')
patch = mpatches.Patch(color='orange', alpha=0.2, label=r'Intervalle de prédiction à {}%'.format(p_int*100))
plt.legend(handles=[mean[0], patch])
ax.set_xlabel('x')
ax.set_ylabel(r'$\hat{y}$')
plt.show()

### 3. Processus Gaussiens

Ci-dessus, on a défini un *a priori* sur les paramètres du modèle linéaire, et donc indirectement sur le modèle en lui-même. Peut-on définir un *a priori* directement sur le modèle ? On va introduire la notion du processus Gaussien, qui est la généralisation d'une distribution Gaussienne multivariée. 

Tout d'abord, un processus aléatoire est une collection de variables aléatoires $Z(x)$, indexées par $x \in \mathcal{X}$. Dans la suite, $\mathcal{X} = \mathbb{R}$. 

Un processus Gaussien $Z$ est un processus aléatoire tel qu'une collection finie de ses variables a une distribution normale multivariée. En d'autres termes, pour une collection d'indices $\{x_1, \ldots, x_N\}$, le vecteur $(Z(x_1), \ldots, Z(x_N))$ est un vecteur Gaussien.

$\longrightarrow$ Une distribution Gaussienne multivariée de dimension $d$ est définie par une moyenne de dimension $d$ et une matrice de covariance de taile $d \times d$. <br>
$\longrightarrow$ Un processus Gaussien $Z$ est défini par une fonction moyenne $m : \mathcal{X} \longrightarrow \mathbb{R}$ et une fonction de covariance $k : \mathcal{X} \times \mathcal{X} \longrightarrow \mathbb{R}$. 

**Processus Gaussien pour la régression**

On va modéliser la relation entre les variables explicatives et la variable à prédire par un processus Gaussien :
$$y_i = Z(x_i) + \epsilon_i \:\:\:\: ; \:\:\:\: \epsilon_i \sim \mathcal{N}(0, \sigma^2)$$

**Exemple de processus Gaussien** 

On fait le choix d'une structure de modèle $\mathcal{M}$ caractérisée par la fonction moyenne et la fonction de covariance :
$$Z \sim GP(m, k) \:\:\:\: \mbox{avec} \:\:\:\: \begin{cases}m(x) = 0  \\ k(x, x') = \mbox{exp}(-\frac{1}{2l}(x - x')^2), l > 0\end{cases}$$


Ainsi, on peut définir une distribution *a priori* sur le modèle de régression, qui s'exprime, pour une collection de points $\boldsymbol{x} = \{x_1, \ldots, x_N\}$, comme suit :
$$p(Z(\boldsymbol{x}) \vert \mathcal{M}) = \mathcal{N}(m(\boldsymbol{x}), k(\boldsymbol{x}, \boldsymbol{x}))$$
où 
$$m(\boldsymbol{x}) = \begin{bmatrix} m(x_1) \\ \vdots \\ m(x_N)\end{bmatrix} \:\:\:\: \mbox{et} \:\:\:\: k(\boldsymbol{x}, \boldsymbol{x}) = \begin{bmatrix} k(x_1, x_1) & k(x_1, x_2) & \ldots & k(x_1, x_N) \\ k(x_2, x_1) & k(x_2, x_2) & \ldots & k(x_2, x_N) \\ \vdots & \vdots & \vdots & \vdots \\ k(x_N, x_1) & k(x_N, x_2) & \ldots & k(x_N, x_N)\end{bmatrix} $$

In [ ]:
class GaussianProcess:
    def __init__(self, mean, cov, noise = 0.1):
        self.mean = mean
        self.cov = cov
        self.noise = noise

    def prior(self, x):
        mean = self.mean(x)
        cov = self.cov(x)
        return np.random.multivariate_normal(mean, cov)

In [ ]:
def zero_mean(x):
    return np.zeros_like(x)

def rbf_kernel(x, scale=0.1):
    return np.exp(- 0.5 * (x[:, np.newaxis] - x[np.newaxis, :])**2 / scale)

In [ ]:
n_samples = 4
fig, ax = plt.subplots(1, 4, figsize=(12, 2))

gp = GaussianProcess(zero_mean, rbf_kernel)

for i in range(n_samples):
    gp_sample = gp.prior(x_test)
    ax[i].plot(x_test, gp_sample, color=colors[i])
    ax[i].set_xlabel('x')
    if i == 0:
        ax[i].set_ylabel('Z(x)')
plt.show()

De manière analogue à la régression Bayésienne, on peut définir la distribution *a posteriori* sur le modèle étant donné les données.

$$p(Z(\boldsymbol{x}^\ast) \vert \boldsymbol{x}, \boldsymbol{y}, \mathcal{M}) = \mathcal{N}(m_{post}(\boldsymbol{x}^\ast), k_{post}(\boldsymbol{x}^\ast, \boldsymbol{x}^\ast))$$

où $m_{post}$ et $k_{post}$ sont obtenus par application du théorème de conditionnement Gaussien.

**Théorème de conditionnement Gaussien**

Soit un vecteur Gaussien de taille $(n_1 + n_2) \times 1$ 
$$\begin{bmatrix} Y_1 \\ Y_2 \end{bmatrix}$$
où $Y_1$ est de taille $n_1 \times 1$ et $Y_2$ est de taille $n_2 \times 1$. On note son vecteur moyen
$$\begin{bmatrix} m_1 \\ m_2 \end{bmatrix}$$
et sa matrice de covariance
$$\begin{bmatrix} \Sigma_1 & \Sigma_{12} \\ \Sigma_{21} & \Sigma_{2} \end{bmatrix}$$
où $\Sigma_1$ est de taille $(n_1 \times n_1)$, $\Sigma_{12}$ est de taille $(n_1 \times n_2)$, $\Sigma_{21}$ est de taille $(n_2 \times n_1)$, et $\Sigma_2$ est de taille $(n_2 \times n_2)$. 

On suppose que $\Sigma_1$ est inversible. Alors, pour tout $y \in \mathbb{R}^{n_1}$, le vecteur aléatoire $Y_2$, conditionnellement à $Y_1 = y_1$, est un vecteur Gaussien avec pour moyenne
$$\mathbb{E}[Y_2 \vert Y_1 = y_1] = m_2 + \Sigma_{12}^T\Sigma_1^{-1}(y_1 - m_1)$$
et pour matrice de covariance
$$\mbox{Cov}[Y_2 \vert Y_1 = y_1] = \Sigma_2 - \Sigma_{12}^T\Sigma_1^{-1}\Sigma_{12}.$$

*Calculer la moyenne de $Y_2$ conditionnellement à $Y_1 = y_1$ si $y_1$ est égal à la moyenne de $Y_1$. Calculer la moyenne et la covariance de $Y_2$ conditionnellement à $Y_1 = y_1$ si $Y_1$ et $Y_2$ sont indépendants. De manière générale, que peut-on dire sur la covariance de $Y_2$ conditionnellement à $Y_1 = y_1$ par rapport à la covariance inconditionnelle de $Y_2$ ?*

Ainsi, la distribution prédictive pour une nouvelle donnée $x^\ast$ est :
$$p(y^\ast \vert x^\ast, \boldsymbol{x}, \boldsymbol{y}, \mathcal{M}) = \mathcal{N}\big(y^\ast; k(x^\ast, \boldsymbol{x})^T(k(\boldsymbol{x}, \boldsymbol{x}) + \sigma^2 \boldsymbol{I})^{-1}\boldsymbol{y}, k(x^\ast, x^\ast) + \sigma^2 -  k(x^\ast, \boldsymbol{x})^T(k(\boldsymbol{x}, \boldsymbol{x}) + \sigma^2 \boldsymbol{I})^{-1}k(x^\ast, \boldsymbol{x})\big)$$

In [ ]:
class GaussianProcess:
    def __init__(self, mean, cov, noise=1e-3):
        self.mean = mean
        self.cov = cov
        self.noise = noise

    def prior(self, x):
        mean = self.mean(x)
        cov = self.cov(x)
        return np.random.multivariate_normal(mean, cov)

    def posterior(self, x1, y1, x2, p_int=0.95):
        n1 = x1.shape[0]
        cov = self.cov(np.concatenate([x1, x2], axis=0))
        cov_1 = cov[:n1, :n1]
        cov_12 = cov[:n1, n1:]
        cov_2 = cov[n1:, n1:]
        prod = np.matmul(cov_12.T, np.linalg.inv(cov_1 + self.noise * np.eye(cov_1.shape[0])))
        posterior_m2 = self.mean(x2) + prod.dot(y1 - self.mean(x1))
        posterior_cov2 = cov_2 + self.noise - prod.dot(cov_12)
        sample = np.random.multivariate_normal(posterior_m2, posterior_cov2)
        Q = norm.isf((1-p_int) / 2)
        l = Q * np.sqrt(np.diag(posterior_cov2) + self.noise)
        low = posterior_m2 - l
        high = posterior_m2 + l
        return posterior_m2, low, high, sample

In [ ]:
gp = GaussianProcess(zero_mean, rbf_kernel)

n_samples = 4
fig, ax = plt.subplots(1, 4, figsize=(12, 2))

x_test = np.linspace(x_train.min() - 0.2, x_train.max() + 0.2, 100)

for i in range(n_samples):
    ax[i].scatter(x_train, y_train)
    mean, low, high, sample = gp.posterior(x_train, y_train, x_test)
    ax[i].plot(x_test, sample, color=colors[i])
    ax[i].set_xlabel('x')
    if i == 0:
        ax[i].set_ylabel('Z(x)')
plt.show()

In [ ]:
x_test = np.linspace(x_train.min() - 1, x_train.max() + 1, 100)
mean, low, high, sample = gp.posterior(x_train, y_train, x_test)

fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(x_train, y_train)
mean = ax.plot(x_test, mean, color='black', label='Espérance')
ax.fill_between(x_test, low, high, alpha=0.2, color='orange')
patch = mpatches.Patch(color='orange', alpha=0.2, label=r'Intervalle de prédiction à {}%'.format(p_int*100))
plt.legend(handles=[mean[0], patch])
ax.set_xlabel('x')
ax.set_ylabel(r'$\hat{y}$')
plt.show()

**Vecteur Gaussien**

Soit $V$ un vecteur aléatoire gaussien dans $\mathbb{R}^n$. Les deux assertions sont équivalentes : 
* Toute combinaison linéaire de $V$ suit une distribution gaussienne. Autrement dit, pour tout vecteur $a$ fixé de dimension $n \times $1, il existe $\mu \in \mathbb{R}$ et $\sigma^2 \geq 0$ tels que $$a^TV = \sum_{i=1}^N a_i V_i \sim \mathcal{N}(\mu, \sigma^2),$$
* Il existe un vecteur $m$ de dimension $n \times 1$, une matrice $K$ de dimension $n \times r$, avec $r \leq n$, et un
vecteur $w$ de dimension $r \times 1$ avec des composantes indépendantes suivant une distribution $\mathcal{N}(0,1)$,
tels que $V = m + Kw$. De plus,
$$\mathbb{E}[V] = m \:\:\:\: \mbox{et} \:\:\:\: \mbox{Cov}(V) = KK^T.$$



**Exercice de cours 1** 

Soient $X_1$ et $X_2$ deux variables aléatoires indépendantes avec une distribution $\mathcal{N}(0, 1)$. Soit
$$Z(x) = X_1 + \mbox{cos}(x)X_2$$
pour $x \in [0, 1]$. Montrer que $Z$ est un processus Gaussien sur $[0,1]$.

**Exercice de cours 2**

On considère trois variables aléatoires indépendantes et identiquement distribuées $A, B, C$ de loi $\mathcal{N}(0, 1)$. 
On pose, pour tout $(t, s) \in [0, 1]^2,$
$$Y(t, s) = tsA + (s − 2t)B + C.$$
* Montrer que $Y$ est un processus Gaussien sur $[0, 1]^2$.
* Quelle est la fonction moyenne de $Y$ ?
* Quelle est la fonction de covariance de $Y$ ?
* Quelle est la loi de $Y(1,1)$ ?
* Quelle est la loi du vecteur $(Y(1,1), Y(0,1))$ conditionnellement à $Y(1, 0) = 2$ ?